In [2]:
import spotchaos.syntheticSignals as sp

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
from matplotlib.animation import TimedAnimation
from mpl_toolkits import mplot3d
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import pickle

import os
from astropy.timeseries import LombScargle
from scipy.integrate import RK45, DOP853
from scipy.stats import chi2, iqr
from scipy.signal import argrelextrema, savgol_filter, find_peaks_cwt
from scipy.spatial.distance import chebyshev
from scipy.optimize import curve_fit

from pytisean import tiseano, tiseanio

from sklearn.neighbors import KDTree, BallTree
from scipy.spatial import KDTree as scipyKDTree

#%matplotlib inline

In [3]:
# Time series:
# 0. White noise
# 1. stationary GP time series
# 2. Simple periodic
# 3. KB88 R(t)
# 4. Rossler x, y, z
# 5. Transformed rossler u, v, w
# 6. Lorenz x, y, z

# and for each we have 3 baselines: TESS (perfect = 257062 data points), Kepler (perfect = 71422 data points), SPECULOOS (perfect = 44119 data points)
#                   and 4 versions: perfect, gapless+noisy, gappy+noiseless, realistic
# yielding 13*3*4 = 156 time series to analyze altogether. (But only 78 files, because the perfect and noisy versions are in the same file.)

# Need to choose:
# 1. time delay (Fraser & Swinney 1986)
# 2. embedding dimension (Cao 1997)
# and then calculate
# 1. correlation dimension (Kurths & Herzel 1987; investigate pytisean too. Look into Thelier window.)
# 2. Lyapunov spectrum (Wolf et al. 1985)
# 3. Lyapunov dimension (Kaplan-Yorke conjecture; see Eckmann & Ruelle 1985 eq 4.11).

# Expected results:
# 0. White noise should have no good choice of time delay; mutual info as a function of delay time should be flat.
# 1. Not sure!
# 2. Simple periodic and KB88 R(t) results should be similar to KB88 figures 9 and 10.
# 3. same
# 4. Rossler x, y, and z should yield time delay ~ 1/4*rossler_qp and embedding dimension of 3. Rossler z results should be worse than x and y.
# 5. Transformed Rossler u, v, and w should yield time delay ~1/4*transformed_rossler_qp and embedding dimension of 3. Variables should be about equally good.
# 6. Lorenz x, y, and z should yield time delay ~1/4*lorenz_qp and embedding dimension of 3. z dimension will be insensitive to wing symmetry.


In [4]:
folderpaths = ['0_gaussian_noise',
               '1_gaussian_process',
               '2_simple_periodic',
               '3_KB88_r',
               '4_rossler',
               '5_transformed_rossler',
               '6_lorenz']

In [5]:
class timeSeriesParams(object):
    def __init__(self, filename, filepath, **kwargs):
        self.filename = filename
        self.filepath = filepath
        self.QPmethod = None
        self.QP = None
        self.noisy_QP = None
        self.bestTauIdx = None
        self.noisy_bestTauIdx = None
        self.sat_m = None
        self.noisy_sat_m = None

        allowed_keys = ["QPmethod","QP","noisy_QP","bestTauIdx","noisy_bestTauIdx","sat_m","noisy_sat_m"]
		
        self.__dict__.update((k,v) for k,v in kwargs.items() if k in allowed_keys)
		
        

In [7]:
# Make a dictionary so we don't have to keep rerunning the FS86 and cao97 steps.
for folder in folderpaths:
    files = os.scandir("../data/{0}/".format(folder))
    for file in files:
        if ".txt" in file.name and "perfect" in file.name and "speculoos" in file.name:
            print(file.name)
            data = np.genfromtxt(file.path)
            
            if "0" in file.name:
                ts = data[:,2]
                noisy_ts = data[:,2]
            else:
                ts = data[:,1]
                noisy_ts = data[:,2]
                
            if "lorenz" in file.name and "_z_" not in file.name:
                QPmethod = "localMaxSep"
            elif "0" in file.name:
                QPmethod = "localMaxSep"
            else:
                QPmethod = "power"

            # choose time delay (Fraser & Swinney 1986)
            if "0" in file.name:
                mutInfo, bestTauIdx, qp = sp.FS86(time = data[:,0], timeSeries = ts, QPmethod=QPmethod, method="global_min", plot=False)
                noisy_mutInfo, noisy_bestTauIdx, noisy_qp = sp.FS86(time = data[:,0], timeSeries = noisy_ts, QPmethod=QPmethod, method="global_min", plot=False)
            else:
                mutInfo, bestTauIdx, qp = sp.FS86(time = data[:,0], timeSeries = ts, QPmethod=QPmethod, method="first_or_second_local_min", plot=False)
                noisy_mutInfo, noisy_bestTauIdx, noisy_qp = sp.FS86(time = data[:,0], timeSeries = noisy_ts, QPmethod=QPmethod, method="first_or_second_local_min", plot=False)
            
            # choose embedding dimension (Cao 1997)
            E1, E2, sat_m, not_change = sp.cao97(timeSeries=ts, tau=bestTauIdx, mMax=8)
            noisy_E1, noisy_E2, noisy_sat_m, not_change = sp.cao97(timeSeries=noisy_ts, tau=noisy_bestTauIdx, mMax=8)
            
            tsp = timeSeriesParams(filename=file.name, filepath=file.path, QPmethod=QPmethod, QP=qp, noisy_QP=noisy_qp, bestTauIdx=bestTauIdx, noisy_bestTauIdx=noisy_bestTauIdx, sat_m=sat_m, noisy_sat_m=noisy_sat_m)
            with open('{0}.pickle'.format(tsp.filename.split(".txt")[0]), 'wb') as f:
                # Pickle the 'data' dictionary using the highest protocol available.
                pickle.dump(tsp, f)

0_gaussianNoise_speculoos_perfect.txt
1_gaussianProcess_speculoos_perfect.txt
2_simplePeriodic_speculoos_perfect.txt
3_KB88r_speculoos_perfect.txt
4_rossler_x_speculoos_perfect.txt
4_rossler_z_speculoos_perfect.txt
4_rossler_y_speculoos_perfect.txt
5_transformed_rossler_z_speculoos_perfect.txt
5_transformed_rossler_x_speculoos_perfect.txt
5_transformed_rossler_y_speculoos_perfect.txt
6_lorenz_x_speculoos_perfect.txt
6_lorenz_z_speculoos_perfect.txt
6_lorenz_y_speculoos_perfect.txt
